# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [47]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,coos bay,43.3665,-124.2179,298.74,72,0,4.63,US,1721345352
1,1,vadso,70.0744,29.7487,285.64,93,100,9.26,NO,1721345353
2,2,hamilton,39.1834,-84.5333,299.20,55,0,2.24,US,1721345196
3,3,guerrero negro,27.9769,-114.0611,296.20,65,0,7.19,MX,1721345058
4,4,yellowknife,62.4560,-114.3525,298.44,50,75,7.20,CA,1721345099


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_1=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity"
    )

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
best_weather = city_data_df.loc[
    ((city_data_df['Max Temp']/10 >= 21.11) & (city_data_df['Max Temp']/10 <= 29.44)) &
    ((city_data_df['Humidity'] >= 40) & (city_data_df['Humidity'] <= 60))
]

# Drop any rows with null values
best_weather_clean=best_weather.dropna()

# Display sample data
best_weather_clean.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,anadyr,64.7500,177.4833,292.43,59,0,4.00,RU,1721345129
31,31,east london,-33.0153,27.9116,290.15,56,100,4.13,ZA,1721345179
48,48,ilulissat,69.2167,-51.1000,285.16,54,40,0.00,GL,1721345163
50,50,dudinka,69.4058,86.1778,281.19,60,0,6.38,RU,1721345092
61,61,nyurba,63.2842,118.3319,289.64,50,40,3.42,RU,1721345423


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = best_weather_clean[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,anadyr,RU,64.7500,177.4833,59,
31,east london,ZA,-33.0153,27.9116,56,
48,ilulissat,GL,69.2167,-51.1000,54,
50,dudinka,RU,69.4058,86.1778,60,
61,nyurba,RU,63.2842,118.3319,50,
64,margaret river,AU,-33.9500,115.0667,51,
106,college,US,64.8569,-147.8028,53,
107,ust-nera,RU,64.5667,143.2000,58,
108,tshabong,BW,-26.0500,22.4500,50,
112,port douglas,AU,-16.4833,145.4667,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [62]:
# Set parameters to search for a hotel
radius = 10000
base_url = "https://api.geoapify.com/v2/places"

print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Define parameters specific to the current row
    params = {
        "filter": f"circle:{longitude},{latitude},{radius}",
        "bias": f"proximity:{longitude},{latitude}",
        "limit": 1,
        "categories": "accommodation.hotel",
        "apiKey": geoapify_key
    }

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

#I had to rearange the code quite a bit to get it to stop throwing errors: moved the params, url

Starting hotel search
anadyr - nearest hotel: Гостевой дом
east london - nearest hotel: No hotel found
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
dudinka - nearest hotel: Виктория
nyurba - nearest hotel: Марина
margaret river - nearest hotel: Margaret River Hotel
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
ust-nera - nearest hotel: гостиница "Солнечная"
tshabong - nearest hotel: No hotel found
port douglas - nearest hotel: Mantra Aqueous Port Douglas
sakaraha - nearest hotel: Palace Club
port augusta - nearest hotel: Hotel Commonwealth
yeppoon - nearest hotel: Blossom's on Seaspray
ulaangom - nearest hotel: АНДУУД
altay - nearest hotel: Sunshine Holiday Hotel
vilyuysk - nearest hotel: Мария
east ballina - nearest hotel: Ballina Beach Resort
alice springs - nearest hotel: Aurora Alice Springs
susuman - nearest hotel: No hotel found
ulladulla - nearest hotel: Hotel Marlin
kabwe - nearest hotel: Tuskers Hotel
chibougamau - nearest hotel: No hotel found
ba

,City,Country,Lat,Lng,Humidity,Hotel Name
10,anadyr,RU,64.7500,177.4833,59,Гостевой дом
31,east london,ZA,-33.0153,27.9116,56,No hotel found
48,ilulissat,GL,69.2167,-51.1000,54,Best Western Plus Hotel Ilulissat
50,dudinka,RU,69.4058,86.1778,60,Виктория
61,nyurba,RU,63.2842,118.3319,50,Марина
64,margaret river,AU,-33.9500,115.0667,51,Margaret River Hotel
106,college,US,64.8569,-147.8028,53,BEST WESTERN PLUS Pioneer Park Inn
107,ust-nera,RU,64.5667,143.2000,58,"гостиница ""Солнечная"""
108,tshabong,BW,-26.0500,22.4500,50,No hotel found
112,port douglas,AU,-16.4833,145.4667,54,Mantra Aqueous Port Douglas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [64]:
%%capture --no-display

# Configure the map plot
map_plot_2=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    hover_cols=["Country","Hotel Name"]
    )

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Hotel Name)

In [54]:
# base_url = "https://api.geoapify.com/v2/places"

# limit=1
# radius=10000
# filters = f"circle:{longitude},{latitude},{radius}"
# bias = f"proximity:{longitude},{latitude}"
# categorie="accommodation.hotel"
# params = {
#     "categories":categorie,
#     "filter":filters,
#     "limit":limit,
#     "bias":bias,
#     "apiKey":geoapify_key    
# }
# h = requests.get(base_url, params=params)
# h = h.json()
# print(h["features"][0]["properties"]["name"])
#this is my test code

Гостевой дом
